In [40]:
import random
from my_lib import *
from IPython.display import clear_output
from datetime import datetime


In [41]:
# Khởi tạo biến toàn cục
dang_nhap_status = False
user = ""

In [42]:
def quay_so():
    danh_sach_giai = {}

    ten_giai = ["GIẢI NHẤT", "GIẢI NHÌ", "GIẢI BA", "GIẢI TƯ", "GIẢI NĂM", "GIẢI SÁU", "GIẢI BẢY"]

    for giai in ten_giai:
        so = ""
        for i in range(5):
            so += str(random.randint(0, 9))  # Tạo một chữ số ngẫu nhiên từ 0 đến 9 và thêm vào số
        danh_sach_giai[giai] = so

    return danh_sach_giai



In [43]:
def in_danh_sach_giai(danh_sach_giai):
    print('-' * 20)
    print("---DANH SÁCH GIẢI---")
    for giai, so in danh_sach_giai.items():
        print(f"{giai}: {so}")
    print('-' * 20)

In [44]:
def nhap_so_lo():
    while True:
        so_lo_input = input("Nhập các số lô bạn muốn chơi, cách nhau bằng dấu phẩy: ")
        danh_sach_so_lo = so_lo_input.split(",")

        so_lo_hop_le = True

        for so_lo in danh_sach_so_lo:
            if len(so_lo) != 2 or not so_lo.isdigit():
                print("Số lô nhập không hợp lệ. Vui lòng nhập lại.")
                so_lo_hop_le = False
                break

        if so_lo_hop_le:
            return danh_sach_so_lo


In [45]:
def nhap_tien_cuoc(username):
    tong_tien = int(lay_thong_tin_tai_khoan(username)[2])
    while True:
        tien_cuoc = input("Nhập tiền cược mà bạn muốn cược (tiền cược < tổng tiền bạn có): ")
        if tien_cuoc.isdigit() and int(tien_cuoc) < tong_tien:
            return int(tien_cuoc)
        else:
            print("Nhập sai định dạng hoặc quá tổng tiền. Nhập lại")

In [46]:

def cap_nhat_tien_choi_lo(username, tong_tien):
    try:
        du_lieu_tai_khoan = doc_file(PATH_DATA_TAI_KHOAN, "r")
        if du_lieu_tai_khoan:
            for i, tai_khoan in enumerate(du_lieu_tai_khoan):
                if tai_khoan[0] == username:
                    du_lieu_tai_khoan[i][2] = str(tong_tien)  # Cập nhật số tiền
                    ghi_file([','.join(tai_khoan) + '\n' for tai_khoan in du_lieu_tai_khoan], PATH_DATA_TAI_KHOAN, "w")
                    return
    except Exception as e:
        print(f"Có lỗi xảy ra khi cập nhật thông tin tài khoản: {e}")


def luu_thong_tin_choi_lo(username, danh_sach_so_lo, tien_cuoc, danh_sach_giai,  tien_thang, tien_thua):
    try:
        thoi_gian_choi = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        thong_tin = f"{thoi_gian_choi},{username},{','.join(danh_sach_so_lo)},{tien_cuoc}," \
                    f"{','.join(danh_sach_giai.values())},{tien_thang},{tien_thua}\n"
        
        ghi_file(thong_tin, PATH_DATA_CHOI_LO, "a")
        print("Thông tin về lần chơi lô đã được lưu.")
        
        # Cập nhật số tiền thắng/thua vào tài khoản

    except Exception as e:
        print(f"Có lỗi xảy ra khi lưu thông tin: {e}")

def game_quay_so(username, danh_sach_so_lo, tien_cuoc):
    tong_tien = int(lay_thong_tin_tai_khoan(username)[2])

    danh_sach_giai = quay_so()
    
    in_danh_sach_giai(danh_sach_giai)
    dem_so_lo_trung = 0
    so_lo_trung = []  # Danh sách lưu trữ số lô trúng

    print("Các số lô bạn chơi là:", danh_sach_so_lo)
    for giai, so_giai in danh_sach_giai.items():
        for so_lo in danh_sach_so_lo:
            if so_lo == so_giai[-2:]:
                dem_so_lo_trung += 1
                so_lo_trung.append(so_lo)
    tien_thang = 0  # Khởi tạo tien_thang
    tien_thua = 0   # Khởi tạo tien_thua

    if dem_so_lo_trung > 0:
        tien_thang = tien_cuoc * dem_so_lo_trung * 70
        tien_thua = tien_cuoc * (len(danh_sach_so_lo) - dem_so_lo_trung)

        tong_tien += tien_thang - tien_thua

        print(f"Bạn đã trúng {dem_so_lo_trung} nháy!")
        print("Các số lô trúng:", ", ".join(so_lo_trung))
        print(f"Số tiền bạn trúng là {tien_thang}")
        print(f"Số tiền bạn thua là {tien_thua}")
        print(f"Tổng tiền bạn có là {tong_tien}")
    else:
        tien_thua = tien_cuoc * len(danh_sach_so_lo)
        tong_tien -= tien_thua

        print("Bạn đã thua lô!")
        print(f"Số tiền bạn thua là {tien_thua}")
        print(f"Tổng tiền bạn còn là {tong_tien}")

    # Lưu thông tin về lần chơi lô vào file
    luu_thong_tin_choi_lo(username, danh_sach_so_lo, tien_cuoc, danh_sach_giai, tien_thang, tien_thua)
    cap_nhat_tien_choi_lo(username, tong_tien)



In [50]:
def main():
    global dang_nhap_status, user
    if dang_nhap_status == False:
        dang_nhap()
    if user == "admin":
        menu_admin()
    else:
        menu()    
    if user == "admin":
        while True:
            lua_chon = int(input("Bạn chọn: "))
            match lua_chon:
                case 1:
                    clear_output(wait=True)
                    while True:
                        username = input("Nhập username cần tạo: ")
                        if kiem_tra_ton_tai_tai_khoan(username) == True:
                            print("Tài khoản user đã tồn tại")
                        elif username == "admin":
                            print("Không được tạo tài khoản admin")
                        else:
                            password = input("Nhập password cần tạo: ")
                            tong_tien = input("Nhập số tiền: ")
                            tao_tai_khoan(username, password, tong_tien)
                        choice = int(input("Bạn có muốn tạo thêm không? (0: không, 1: có)"))
                        if choice == 0:
                            break
                    clear_output()
                    main()
                    break
                case 2:
                    clear_output(wait=True)
                    while True:
                        username = input("Nhập username cần xóa: ")
                        if kiem_tra_ton_tai_tai_khoan(username) == False:
                            print("Tài khoản user không tồn tại")
                        elif username == "admin":
                            print("Không được xóa tài khoản admin")
                        else:
                            xoa_tai_khoan(username)
                        choice = int(input("Bạn có muốn xóa thêm không? (0: không, 1: có)"))
                        if choice == 0:
                            break
                    clear_output()
                    main()
                    break
                case 3:
                    clear_output(wait=True)
                    while True:
                        username = input("Nhập username cần nạp tiền: ")
                        if kiem_tra_ton_tai_tai_khoan(username) == False:
                            print("Tài khoản user không tồn tại")
                        else:
                            so_tien_nap = int(input("Nhập số tiền cần nạp: "))
                            nap_tien(username, so_tien_nap)
                        choice = int(input("Bạn có muốn nạp thêm không? (0: không, 1: có)"))
                        if choice == 0:
                            break
                    clear_output()
                    main()
                    break
                case 4:
                    clear_output()
                    while True:
                        thong_ke_admin()
                        choice = input("Nhấn Enter để quay lại menu: ")
                        if choice == "":
                            break
                    clear_output()
                    main()
                    break
                case 5:
                    dang_nhap_status = False
                    print("Chào bạn, hẹn gặp lại!")
                    break
                case _:
                    print("Nhập sai lựa chọn, vui lòng nhập lại !")
    else:
        while True:
            lua_chon = int(input("Bạn chọn: "))
            match lua_chon:
                case 1:
                    clear_output(wait=True)
                    while True:
                        tien_cuoc = nhap_tien_cuoc(user)
                        danh_sach_so_lo = nhap_so_lo()
                        game_quay_so(user, danh_sach_so_lo, tien_cuoc)
                        choi_tiep = int(input("Bạn có muốn chơi tiếp không? (0: không, 1: có)"))
                        if choi_tiep == 0:
                            break
                    clear_output()
                    main()
                    break
                case 2:
                    clear_output(wait=True)
                    while True:
                        password_cu = input("Nhập password cũ của bạn: ")
                        if kiem_tra_tai_khoan(user,password_cu) == "":
                            print("Sai password !")
                            break
                        else:
                            password_moi = input("Nhập password mới của bạn: ")
                            doi_mat_khau(user, password_cu, password_moi)
                        choice = int(input("Bạn có muốn đổi mật khẩu khác không? (0: không, 1: có)"))
                        if choice == 0:
                            break
                    clear_output()
                    main()
                    break
                case 3:
                    clear_output()
                    while True:
                        thong_ke_user(user)
                        choice = input("Nhấn Enter để quay lại menu: ")
                        if choice == "":
                            break
                    clear_output()
                    main()
                    break
                   
                case 4:
                    dang_nhap_status = False
                    print("Chào bạn, hẹn gặp lại!")
                    break
                case _:
                    print("Nhập sai lựa chọn, vui lòng nhập lại !")


In [51]:
main()

GAME LÔ ĐỀ HỌC
1. Chơi lô
2. Đổi mật khẩu
3. Thống kê
4. Đăng xuất
Chào bạn, hẹn gặp lại!
